In [6]:
import pandas as pd
from decimal import Decimal

# Načte prvních 200 řádků
df = pd.read_excel(".\Eurotec.xlsx", nrows=200, engine="openpyxl")

# Najde první prázdný řádek
first_empty_index = df[df.isnull().all(axis=1)].index.min()

# Ořízne tabulku, pokud takový řádek existuje
if pd.notna(first_empty_index):
    df = df.loc[:first_empty_index - 1]

# Odstraní prázdné sloupce
df = df.dropna(axis=1, how='all')

# Přejmenování názvů sloupců
df.rename(columns={
    "Unnamed: 6": "Kopf",
    "Unnamed: 7": "Abmessung",
}, inplace=True)

# Rozdělení sloupce Abmessung - rozměr na průměr a délka a převod na Decimal
def rozdel_rozmer_a_preved_na_decimal(row):
    try:
        prumer_str, delka_str = row['Abmessung'].replace(',', '.').split('x')
        return Decimal(prumer_str.strip()), Decimal(delka_str.strip())
    except Exception:
        return None, None

df[['prumer', 'delka']] = df.apply(
    lambda row: pd.Series(rozdel_rozmer_a_preved_na_decimal(row)), axis=1
)

# Odstranění nepotřebných sloupců
df.drop(columns=['Unnamed: 0', 'Abmessung', 'Gew + Tara', 'VPE', 'Box', 'Anzahl Boxen pro Behälter', 'Gew.', 'Härterei',
                 'von Härterei \nnach Galvanik', 'Galvanik', 'vom Galvanik nach Eurotec', 'Prod. Datum'], inplace=True, errors='ignore')

column_mapping = {
    'Abhol- datum': 'datum',
    'Material- charge': 'sarze',
    'Vorgang+': 'prubeh',
    'Artikel- nummer': 'artikl',
    'Kopf': 'typ_hlavy',
    'Be-schich-tung': 'vrstva',
    'Bezeichnung': 'popis',
    'n. Zg. / \nas drg': 'predpis',
    'Material': 'material',
    'Ober- fläche': 'povrch',
    'Gewicht in kg': 'hmotnost',
    'Tara kg': 'tara',
    'Menge       ': 'mnozstvi',
    'Behälter-Nr.:': 'behalter_nr',
    'Sonder / Zusatzinfo': 'dodatecne_info',
    'Lief.': 'dodavatel_materialu',
    'Fertigungs- auftrags Nr.': 'vyrobni_zakazka'
}

df.rename(columns=column_mapping, inplace=True)

df.columns.tolist()

['datum',
 'sarze',
 'prubeh',
 'artikl',
 'typ_hlavy',
 'vrstva',
 'popis',
 'predpis',
 'material',
 'povrch',
 'hmotnost',
 'tara',
 'mnozstvi',
 'behalter_nr',
 'dodatecne_info',
 'dodavatel_materialu',
 'vyrobni_zakazka',
 'prumer',
 'delka']

In [7]:
df

,datum,sarze,prubeh,artikl,typ_hlavy,vrstva,popis,predpis,material,povrch,hmotnost,tara,mnozstvi,behalter_nr,dodatecne_info,dodavatel_materialu,vyrobni_zakazka,prumer,delka
0,2025-06-03,420303,30625.0,944722,SK,5-8,HSPT SK DAG T40 8x220/95,1052.0,19MnB4,blau,320.0,61.0,NaN,37.0,muss entphosphatiert werden,FN,02510-01512,8,220
1,2025-06-03,420352,30625.0,944721,SK,5-8,HSPT SK DAG T40 8x200/95,1052.0,19MnB4,blau,323.0,61.0,NaN,43.0,muss entphosphatiert werden,FN,02510-01512,8,200
2,2025-06-03,619628,30625.0,HBU944736,SK,5-8,HSPT SK AG T40 8x500/95,1052.0,19MnB4,blau,329.0,61.0,3163.461538,67.0,müssen gerichtet werden,FN,02510-01139,8,500
3,2025-06-03,420303,30625.0,944721,SK,5-8,HSPT SK DAG T40 8x200/95,1052.0,19MnB4,blau,145.0,61.0,NaN,72.0,muss entphosphatiert werden,FN,02510-01512,8,200
4,2025-06-03,T48456-1,30625.0,944721,SK,5-8,HSPT SK DAG T40 8x200/95,1052.0,19MnB4,blau,303.0,61.0,NaN,84.0,muss entphosphatiert werden,VD,02510-01512,8,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,2025-06-03,420303,30625.0,944721,SK,5-8,HSPT SK DAG T40 8x200/95,1052.0,19MnB4,blau,323.0,63.0,NaN,2895.0,muss entphosphatiert werden,FN,02510-01512,8,200
70,2025-06-03,420303,30625.0,944722,SK,5-8,HSPT SK DAG T40 8x220/95,1052.0,19MnB4,blau,321.0,63.0,NaN,2920.0,muss entphosphatiert werden,FN,02510-01512,8,220
71,2025-06-03,420352,30625.0,945800,ZK,5-8,FB HSPT ZK DAG T40 8x380/95,1053.0,19MnB4,blau,177.0,63.0,NaN,2947.0,muss entphosphatiert werden,FN,02510-00074,8,380
72,2025-06-03,420303,30625.0,944602,TK,5-8,HSPT TK DAG T40 8x360/95,1051.0,19MnB4,blau,184.0,63.0,NaN,2952.0,muss entphosphatiert werden,FN,02510-00606,8,360
